# Finetune a text classification model
In this guide, we will show how to finetune a `DistilledBert` model to classify SMS as spam or not. We will also show how we can use MLflow to track and monitor our finetuning. We will be using [Databricks Community Edition](https://community.cloud.databricks.com/) for visualization, it is completely free for use. If you haven't, you can register an account now via [link](https://www.databricks.com/try-databricks), or we will come back to it later.

We recommend turn on the free-tier GPU in Colab by **Edit -> notebook settings -> Hardware Accelerator**, it will significanly shorten the finetuning time.

## Install dependencies

We need `transformers`, `datasets` and `evaluate` package by Huggingface. Additionally, we install `mlflow` for tracking purpose.

In [1]:
%pip install -q mlflow datasets evaluate transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta

We also need to update the `accelerate` package to be compatible with the training.

In [2]:
%pip install -U -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.9 MB/s eta 0:00:00


In [3]:
# There is a strange pydantic issue, we can delete this in the future.
%pip install pydantic==1.10.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


Make sure you click on Runtime -> Restart runtime after installation to reflect the package change.


## Load the dataset

In this guide we will load the SMS Spam Collection dataset from HuggingFace: [link](https://huggingface.co/datasets/sms_spam). Each record of the dataset consists of a message and a label (spam or not). HuggingFace provides a nice [preview]((https://huggingface.co/datasets/sms_spam)) on the dataset.

In [ ]:
from datasets import load_dataset

# Load "sms_spam" dataset.
sms_dataset = load_dataset("sms_spam")

# Split train/test by 8:2.
sms_train_test = sms_dataset["train"].train_test_split(test_size=0.2)
train_dataset = sms_train_test["train"]
test_dataset = sms_train_test["test"]

## Data preprocessing

Let's do some data preprocessing before finetuning, we will do:
1. Tokenize the string data into a list of ints (token ids).
2. Pad the tokenized data to the same length, which is 128 tokens in this guide.
3. Shuffle the dataset.

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for "distilbert-base-uncased" model.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def tokenize_function(examples):
    # Pad/truncate each text to 512 tokens. Enforcing the same shape
    # could make the training faster.
    return tokenizer(
        examples["sms"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )


train_tokenized = train_dataset.map(tokenize_function)
train_tokenized = train_tokenized.remove_columns(["sms"]).shuffle(seed=42)

test_tokenized = test_dataset.map(tokenize_function)
test_tokenized = test_tokenized.remove_columns(["sms"]).shuffle(seed=42)

## Set up finetuning pipeline

Now let's go ahead setting up our finetuning pipeline. We will use HuggingFace `Trainer` API to finetune our model.

Let's load `DistilledBert` model from HuggingFace. We will use class `AutoModelForSequenceClassification`, which basically gives a DistilledBert model plus a fully connected layer to map the feature representation to probability distribution over our classes.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Set the mapping between int label and its meaning.
id2label = {0: "ham", 1: "spam"}
label2id = {"ham": 0, "spam": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    label2id=label2id,
    id2label=id2label,
)

Create the evaluation metric to log. Loss is also logged, but adding other metrics such as accuracy can make modeling performance easier to understand. For classification task, we use `accuracy` as the tracking metric.

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Let's set up the trainer! We need to configure two things:
- `TrainingArguments`: it sets hyperparameters like batch size, logging frequency and so. Please refer to [transformers documentation](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)
for the full arg list. Don't panick on the long list of args, usually we just need a few out of that.
- `Trainer`: `Trainer` hooks things together, including the model, training args, training/evaluation dataset, and evaluation function.

In [8]:
from transformers import Trainer, TrainingArguments

# Checkpoints will be output to this `training_output_dir`.
training_output_dir = "sms_trainer"
training_args = TrainingArguments(
    output_dir=training_output_dir,
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=8,
    num_train_epochs=3,
)

# Put things together with a `Trainer` instance.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

We are ready for kicking off the training! Before that, let's set up our tracking/visualization tool - MLflow + Databricks (Community Edition).

## Set up tracking/visualization tool

If you have not, please register an account of [Databricks community edition](https://www.databricks.com/try-databricks#account). It should take no longer than 1min to register.

Databricks CE (community edition) is a free platform for users to try out Databricks features. For this guide, we need the ML experiment dashboard for us to track our training progress.

After you have sucessfully registered an account, all you need to do is to run the command below to connect from Google Colab to your Databricks account. You will need to enter following information at prompt:
- **Databricks Host**: https://community.cloud.databricks.com/
- **Username**: your signed up email
- **Password**: your password

In [9]:
!databricks configure

Databricks Host (should begin with https://): https://community.cloud.databricks.com/
Username: qianchen94era@gmail.com
Password: 
Repeat for confirmation: 


If you have access to Databricks production version, you will need to run `!databricks configure --token` instead, and generate a **personal access token** in your account and paste it. Other setups are the same.

Now this colab is connected to the hosted tracking server. Let's configure MLflow metadata. Two things to set up:
- `mlflow.set_tracking_uri`: always use "databricks".
- `mlflow.set_experiment`: pick up a name you like, start with `/`

In [10]:
import mlflow

# This is always "databricks" when using a databricks hosted tracking server.
mlflow.set_tracking_uri("databricks")
# Pick up a name you like.
mlflow.set_experiment("/finetune-a-spam-classifier")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3039554150514788', creation_time=1692052967482, experiment_id='3039554150514788', last_update_time=1692218287414, lifecycle_stage='active', name='/finetune-a-spam-classifier', tags={'mlflow.experiment.sourceName': '/finetune-a-spam-classifier',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'qianchen94era@gmail.com',
 'mlflow.ownerId': '3209978630771139'}>

## Kick off the finetuning

We have every piece! Now let's kick off the finetuning.

In [11]:
with mlflow.start_run() as run:
    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.193300,0.050481,0.987444
2,0.000800,0.029386,0.993722
3,0.000200,0.042648,0.993722


While your training is ongoing, you can find this training in your dashboard. Log in to your [Databricks CE](https://community.cloud.databricks.com/) account, and click on top left to select machine learning in the drop down list. Then click on the experiment icon. See the screenshot below:
![landing page](https://drive.google.com/uc?export=view&id=1QxVaolr-L-w96pKUOiYQut3aSRE-04tC)

After clicking the `Experiment` button, it will bring you to the experiment page, where you can find your runs. Clicking on the most recent experiment and run, you can find your metrics there, similar to:
![experiment page](https://drive.google.com/uc?export=view&id=1M-oycljsFAHBVip81Rprwx57Ape1uCq-)

You can click on metrics to see the chart.

## Evaluate the finetuned model

Now we have finished the finetuning, let's see how it works. We will wrap the finetuned model into a HuggingFace `text-classification` pipeline. Using pipeline is optional, but it has several benefits including it automatically does tokenization and detokenization, so we can feed a string input and get a string output.

In [12]:
from transformers import pipeline

pipe = pipeline(
    "text-classification",
    model=trainer.model,
    batch_size=8,
    tokenizer=tokenizer,
    device=0,
)

# Make a random message.
sample_text = (
    "WINNER!! As a valued network customer you have been selected "
    "to receivea £900 prize reward! To claim call 1234567. Claim "
    "code GEEZ. Valid 12 hours only."
)
print("Model prediction: ", pipe(sample_text))

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Model prediction:  [{'label': 'spam', 'score': 0.999816358089447}]


That's it! You have made it, thanks for reading!